# Prevalence inference
Based on the [paper](http://www.sciencedirect.com/science/article/pii/S1053811916303470) and [code](https://github.com/allefeld/prevalence-permutation) by Allefeld, Gorgen, & Haynes.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Data

In [16]:
N_subjects = 20
K_voxels = (9, 10, 9)
P1_perms = 1000

perms = np.random.normal(loc=0.5, scale=0.05, size=(N_subjects, np.prod(K_voxels), P1_perms))
obs = np.random.normal(loc=0.55, scale=0.05, size=(N_subjects, np.prod(K_voxels)))
all_data = np.dstack((obs[:, :, np.newaxis], perms))

print("Perms shape: %s" % (perms.shape,))
print("Observed values shape: %s" % (obs.shape,))
print("All_data shape: %a" % (all_data.shape,))

Perms shape: (20, 810, 1000)
Observed values shape: (20, 810)
All_data shape: (20, 810, 1001)


## Analysis params

In [4]:
P2_perms = 1000
alpha = 0.05

## Analysis

In [6]:
minstat_L2 = np.min(obs, axis=0)

In [ ]:
# Find second level stats

for i in range(P2_perms):
    